# Benchmarks

## Initialize

In [9]:
#library(Rmisc)
library(dtplyr)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)
library(data.table)
library("jsonlite")
library(ggthemes)

Warning message:
“package ‘tidyverse’ was built under R version 4.0.3”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.0.5”
Warning message:
“package ‘readr’ was built under R version 4.0.5”
Warning message:
“package ‘purrr’ was built under R version 4.0.3”
Warning message:
“package ‘dplyr’ was built under R version 4.0.5”
Warning message:
“package ‘stringr’ was built under R version 4.0.5”
Warning message:
“package ‘forcats’ was built under R version 4.0.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘arrow’ was built under R version 4.0.3”

Attaching package: ‘arrow’


The following object is masked from ‘p

In [10]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

project_label="22_medical_records"
project_path = glue("{base_path}/results/projects/{project_label}")
figure_path = glue("{project_path}/figures")
output_path = glue("{project_path}/data")

experiment = 220413
experiment_path = glue("{output_path}/{experiment}")

[1] "/sc-projects/sc-proj-ukb-cvd"


In [11]:
endpoints_md = fread(glue("{experiment_path}/endpoints.csv"))
endpoints = sort(endpoints_md$endpoint)

In [12]:
endpoints_md

V1,endpoint,n,freq
<int>,<chr>,<int>,<dbl>
0,OMOP_4306655,39253,0.0781227299
1,phecode_001,308,0.0006148453
2,phecode_002,1481,0.0029526578
3,phecode_002-1,1443,0.0028764674
4,phecode_003,2875,0.0057494940
5,phecode_004,2061,0.0041365607
6,phecode_004-1,761,0.0015177109
7,phecode_005,652,0.0013054308
8,phecode_005-1,532,0.0010650885


In [13]:
endpoint_defs = arrow::read_feather(glue("{output_path}/phecode_defs_220306.feather")) %>% arrange(endpoint)

In [14]:
name = "benchmark_cindex_agesexcoxph_220421"
benchmark_endpoints = arrow::read_feather(glue("{experiment_path}/{name}.feather")) %>% left_join(endpoint_defs) 

Joining, by = "endpoint"



In [15]:
benchmark_clean = benchmark_endpoints %>% 
    filter(score %in% c("Age+Sex", "Age+Sex+MedicalHistory")) %>% 
    pivot_wider(names_from="score", values_from="cindex") %>% 
    mutate(delta=`Age+Sex+MedicalHistory`-`Age+Sex`)
benchmark_agg = benchmark_clean %>% group_by(endpoint) %>% summarise(across(all_of(c("Age+Sex+MedicalHistory", "Age+Sex", "delta")), ~ median(.)))

In [16]:
do_md = endpoints_md %>% #mutate(endpoint = as.character(endpoint)) %>%
    left_join(endpoint_defs) %>% 
    left_join(benchmark_agg) %>% as_tibble()

Joining, by = "endpoint"

Joining, by = "endpoint"



[1] "Death"          "ID"             "Neoplasms"      "Blood"         
 [5] "Endo"           "Metab"          "Mental"         "Neuro"         
 [9] "Eye"            "Hearing"        "Cardio"         "Resp"          
[13] "GI"             "Genitourinary"  "Derm"           "Musc/Skel"     
[17] "Signs/Symptoms" "Preg"           "Rx"             "Stat"

In [17]:
do_md %>% arrange(delta) %>% filter(phecode=="105.1")

V1,endpoint,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf,Age+Sex+MedicalHistory,Age+Sex,delta
<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>
96,phecode_105-1,8730,0.03288359,105.1,"Malignant neoplasm of the breast, female",Neoplasms,Female,0,105,1,0.6263401,0.534015,0.09232658


In [18]:
do_md %>% filter(is.na(delta)) %>% arrange(delta)

V1,endpoint,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf,Age+Sex+MedicalHistory,Age+Sex,delta
<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>


In [29]:
do_md %>% 
    #filter(str_detect(phecode_category, "Neoplasm")) %>%
    filter(str_detect(phecode_string, "parkin"))
    #arrange(desc(ratio))

V1,endpoint,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf,Age+Sex+MedicalHistory,Age+Sex,delta
<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>
429,phecode_324-12,188,0.0003741815,324.12,Secondary parkinsonism,Neuro,Both,0,324,1,0.8325482,0.6617301,0.17225715
430,phecode_324-2,254,0.0005055239,324.2,Degenerative diseases of the basal ganglia (excluding parkinsons),Neuro,Both,0,324,0,0.7448947,0.7259771,0.01390294


In [51]:
do_md %>% 
    filter(str_detect(phecode_category, "Cardio")) %>%
    filter(!str_detect(phecode, "\\."))# %>%
#filter(freq>0.001) %>%
    #filter(str_detect(phecode_string, "Embolism")) %>%
    #sample_n(10) 
    #arrange(desc(ratio))

V1,endpoint,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf,Age+Sex+MedicalHistory,Age+Sex,delta
<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>
745,phecode_400,750,0.0014967909,400,Rheumatic fever and chronic rheumatic heart diseases,Cardio,Both,0,400,0,0.7888950,0.6932507,0.09470846
747,phecode_401,99768,0.2458042199,401,Hypertension,Cardio,Both,0,401,0,0.6706964,0.6319888,0.03868927
751,phecode_402,27836,0.0588967128,402,Elevated blood pressure reading without diagnosis of hypertension,Cardio,Both,0,402,1,0.6708653,0.5291183,0.14159333
752,phecode_403,12185,0.0248343021,403,Angina pectoris,Cardio,Both,0,403,0,0.7954763,0.6777273,0.11779154
754,phecode_404,16554,0.0338738886,404,Ischemic heart disease,Cardio,Both,0,404,0,0.7780005,0.6841571,0.09380147
758,phecode_406,782,0.0015567053,406,Chronic pulmonary heart disease,Cardio,Both,0,406,0,0.8482223,0.7018413,0.14735210
760,phecode_410,3631,0.0072834278,410,Inflammation of the heart [Carditis],Cardio,Both,0,410,0,0.7160960,0.6285403,0.08766039
764,phecode_411,562,0.0011187241,411,Other diseases of pericardium,Cardio,Both,0,411,0,0.7428952,0.5591639,0.18346230
766,phecode_413,10233,0.0205376765,413,Heart valve disorders,Cardio,Both,0,413,0,0.7736118,0.6915581,0.08232596


In [49]:
do_md %>% 
    filter(str_detect(phecode_category, "Neuro")) %>%
    filter(str_detect(phecode, "452"))# %>%
#filter(freq>0.001) #%>%
    #filter(str_detect(phecode_string, "Embolism")) %>%
    #sample_n(10) 
    #arrange(desc(ratio))

V1,endpoint,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf,Age+Sex+MedicalHistory,Age+Sex,delta
<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>


In [27]:
unique(do_md$phecode_category)

[1] "Death"          "ID"             "Neoplasms"      "Blood"         
 [5] "Endo"           "Metab"          "Mental"         "Neuro"         
 [9] "Eye"            "Hearing"        "Cardio"         "Resp"          
[13] "GI"             "Genitourinary"  "Derm"           "Musc/Skel"     
[17] "Signs/Symptoms" "Preg"           "Rx"             "Stat"

In [52]:
endpoint_selection = do_md %>% filter(phecode_string %in% c("Anemia", 
                                       "Ischemic heart disease",
                                       #"Stroke and transient cerebral ischemic attacks",
                                       #"Essential hypertension",
                                       "Fibrosis and cirrhosis of liver",
                                        "Malignant neoplasm of the prostate",
                                       #"Hypertensive heart disease",
                                       #"Hypertensive chronic kidney disease",
                                       "Ischemic heart disease",
                                       "Deep vein thrombosis [DVT]", 
                                                            "Psoriasis",
                                                            "Parkinson's disease (Primary)",
                                       #"Renal failure",
                                       #"Venous thromboembolism", 
                                       #"Hypo-coagulability",
                                       #"Endocarditis", 
                                       "Heart failure",
                                       "All-Cause Death",
                                        "Type 2 diabetes",
                                        "Inflammatory bowel disease",
                                                            "Hypertension",
                                       "Pneumonia",
                                       #"Pyelonephritis",
                                       #"Hyperglyceridemia",
                                        "Helicobacter [H. pylori]",
                                       "Chronic kidney disease",
                                        "Pyelonephritis",
                                                            "Somatoform disorders",
                                       #"Urinary tract infection",
                                       #"Preeclampsia and eclampsia",
                                       #"Acute respiratory infection",
                                       "Rheumatoid arthritis",
                                       #"Vitamin deficiencies",
                                       #"Hypothyroidism",
                                       #"Influenza virus",
                                       #"Pseudomonas",
                                       #"Atrial fibrillation",
                                        "Suicide ideation and attempt or self harm",
                                       "Sepsis",
                                       "Malignant neoplasm of the breast",
                                       #"Meningitis",
                                       "Chronic obstructive pulmonary disease [COPD]",
                                       "Adverse effect of anesthesia",
                                        #"Somatoform disorder",
                                      #"Sleep apnea",
                                       #"Crohn's disease",
                                       #"Ulcerative colitis",
                                       #"Chronic pancreatitis",
                                       #"Multiple myeloma and malignant plasma cell neoplasms",
                                       #"Appendicitis",
                                       # "Embolism and thrombosis",
                                      "Dementias and cerebral degeneration",
                                                            "Cataract"
                                      #"Blindness and low vision"
                                      )) %>%
    arrange(as.numeric(phecode))
endpoint_selection

V1,endpoint,n,freq,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf,Age+Sex+MedicalHistory,Age+Sex,delta
<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>
11,phecode_008,3253,0.006529375,008,Helicobacter [H. pylori],ID,Both,0,008,1,0.7225489,0.5169990,0.2050851628
58,phecode_092-2,4397,0.008766089,092.2,Sepsis,ID,Both,0,092,0,0.7738389,0.6349941,0.1386551254
95,phecode_105,9100,0.018416952,105,Malignant neoplasm of the breast,Neoplasms,Both,0,105,0,0.7958123,0.7472772,0.0488502983
106,phecode_107-2,13126,0.058125426,107.2,Malignant neoplasm of the prostate,Neoplasms,Male,0,107,1,0.6951852,0.6886326,0.0065931571
206,phecode_164,24201,0.050355491,164,Anemia,Blood,Both,0,164,0,0.7450362,0.5653100,0.1796375944
273,phecode_202-2,30247,0.062832369,202.2,Type 2 diabetes,Endo,Both,0,202,1,0.7760098,0.5853683,0.1905535242
376,phecode_284,4128,0.008265886,284,Suicide ideation and attempt or self harm,Mental,Both,0,284,0,0.8495307,0.6207862,0.2292363562
400,phecode_292,632,0.001262765,292,Somatoform disorders,Mental,Both,0,292,0,0.7743454,0.5883394,0.1873185463
428,phecode_324-11,4196,0.008363531,324.11,Parkinson's disease (Primary),Neuro,Both,0,324,1,0.7378301,0.7384004,-0.0001054323


In [53]:
print(endpoint_selection$endpoint)

 [1] "phecode_008"    "phecode_092-2"  "phecode_105"    "phecode_107-2" 
 [5] "phecode_164"    "phecode_202-2"  "phecode_284"    "phecode_292"   
 [9] "phecode_324-11" "phecode_328"    "phecode_371"    "phecode_401"   
[13] "phecode_404"    "phecode_424"    "phecode_440-11" "phecode_468"   
[17] "phecode_474"    "phecode_522-1"  "phecode_542-1"  "phecode_581-1" 
[21] "phecode_583"    "phecode_665"    "phecode_705-1"  "OMOP_4306655"  


In [160]:
unique(do_md$phecode_category)

[1] "Death"          "ID"             "Neoplasms"      "Blood"         
 [5] "Endo"           "Metab"          "Mental"         "Neuro"         
 [9] "Eye"            "Hearing"        "Cardio"         "Resp"          
[13] "GI"             "Genitourinary"  "Derm"           "Musc/Skel"     
[17] "Signs/Symptoms" "Preg"           "Rx"             "Stat"

In [142]:
do_md %>% 
    #filter(leaf==0) %>%
    filter(delta<0.02) %>%
    arrange(delta)# %>% 
    #filter(str_detect(phecode_string, "ardio"))
    #filter(phecode_category == "Resp")

endpoint,ratio,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf,Age+Sex+MedicalHistory,AgeSex,delta
<glue>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>
phecode_619-32,0.0006424947,619.32,Erosion and ectropion of cervix,Genitourinary,Female,0,619,1,0.6463398,0.7875073,-0.1411674543
phecode_904,0.0001941697,904,Bleeding in pregnancy,Preg,Female,0,904,0,0.7655653,0.8974697,-0.1319044085
phecode_342,0.0013188065,342,Plegia and unspecified paralysis,Neuro,Both,0,342,0,0.4280804,0.5260091,-0.0979286823
phecode_901-3,0.0005405189,901.3,Missed abortion,Preg,Female,0,901,1,0.6323496,0.7274490,-0.0950993962
phecode_106-13,0.0007812719,106.13,Malignant neoplasm of the cervix,Neoplasms,Female,0,106,1,0.5474454,0.6111153,-0.0636699071
phecode_500,0.0023352387,500,Disorders of tooth develoment,GI,Both,0,500,0,0.5664366,0.6141913,-0.0477547481
phecode_106-3,0.0029915356,106.3,Malignant neoplasm of the ovary,Neoplasms,Female,0,106,1,0.5823450,0.6263405,-0.0439955553
phecode_614-3,0.0011298192,614.3,Pelvic inflammatory disease (PID),Genitourinary,Female,0,614,0,0.6377704,0.6811828,-0.0434123906
phecode_200-31,0.0014710221,200.31,Graves' disease [Toxic diffuse goiter],Endo,Both,0,200,1,0.6210898,0.6637040,-0.0426142725
